# The demonstration of the tropical precipitation module on the different datasets

## Structure
 1) Load of  packages, modules and data
 2) Diagnostic with manual data preprocessing
 3) Diagnostic with automatic data preprocessing
 4) Calculation time expectation  
 5) Plots and animations

## 1) Load of  packages, modules and data

###  Loading the necessary packages

In [1]:
import dask

import re
import matplotlib as mpl
# Define Agg as Backend for matplotlib when no X server is running
mpl.use('Agg')
#import socket
import os
import importlib

#import inspect
#import timeit
import sys

### Storing the path to the current repository into the variable

In [2]:
with os.popen("pwd ") as f:
    _pwd = f.readline()

pwd = re.split(r'[\n]', _pwd)[0]

### Setting the path to the diagnostic repository 

In [3]:
sys.path.append(str(pwd)+'/../')
import src.shared_func
import src.tr_pr_mod

### Loading the extra function, which can be used in any diagnostic

In [4]:
importlib.reload(src.shared_func)
from  src.shared_func import time_interpreter,  animation_creator, image_creator,  xarray_attribute_update,  data_size

### Loading the tropical precipitation module

In [5]:
importlib.reload(src.tr_pr_mod)
from  src.tr_pr_mod import TR_PR_Diagnostic

### Importing the aqua module

In [ ]:
import aqua

In [ ]:
from aqua import Reader
from aqua.reader import catalogue

In [ ]:
configdir = '../../../config/'
diagname  = 'tr_pr'
machine   = 'levante'
catalogue(configdir=configdir)

### IFS data

In [ ]:
reader = Reader(model="IFS", exp="tco3999-ng5", source="ICMGG_atm2d",configdir=configdir, regrid="r200")
IFS = reader.retrieve()

In [ ]:
IFS

### Selecting the small part of dataset for tests

In [ ]:
IFS_chunk_regrided = reader.regrid(IFS["tprate"][10:15,:])

In [ ]:
IFS_chunk_regrided 

## 2) Diagnostic with manual data preprocessing

### Initializing the class object and Setting the class attributes

In [7]:
diag = TR_PR_Diagnostic()

# mandatory attributes
diag.num_of_bins = 15
diag.first_edge = 0
diag.width_of_bin = 1*10**(-4)/diag.num_of_bins

# other attributes of class whoch have a default value
diag.trop_lat = 10
diag.i_timestep = None
diag.f_timestep = None
diag.s_year = None
diag.f_year = None 
diag.s_month = None
diag.f_month = None 

### OR

In [8]:
diag = TR_PR_Diagnostic(num_of_bins = 15, first_edge = 0, width_of_bin = 1*10**(-6)/15)

### Original coordinates

In [14]:
diag.coordinate_names(IFS)

('lat', 'lon')

### Size of entire dataset

In [15]:
data_size(IFS)

1732208720000

### Timestep of dataset is

In [16]:
time_interpreter(IFS)

'1H'

### Coordinates after regrid

In [19]:
diag.coordinate_names(IFS_chunk_regrided)

('lat', 'lon')

### Size of selected chunk

In [20]:
data_size(IFS_chunk_regrided)

81000

### Attribute update

In [21]:
IFS_chunk_regrided = xarray_attribute_update(IFS_chunk_regrided, IFS)

### Load selected chunk into a memory

In [22]:
IFS_chunk_regrided = IFS_chunk_regrided.compute()

### Units of precepitation rate

In [23]:
IFS_chunk_regrided.attrs['units']

'm s**-1'

In [24]:
time_interpreter(IFS_chunk_regrided) 

'1H'

### Tropical latitudes

In [25]:
IFS_chunk_regrided_trop = diag.ds_per_lat_range(IFS_chunk_regrided)

In [ ]:
IFS_chunk_regrided_trop

In [ ]:
#IFS_chunk_regrided_trop = IFS_chunk_regrided_trop.compute()

### Size reduction

In [27]:
data_size(IFS_chunk_regrided)

81000

In [28]:
data_size(IFS_chunk_regrided_trop)

9000

### Fastest histogram

In [29]:
hist_IFS  = diag.hist1d_fast(IFS_chunk_regrided,  preprocess = False)
hist_IFS

<xarray.DataArray (bin: 15)>
array([7.3648e+04, 2.9230e+03, 1.3840e+03, 8.4800e+02, 5.3300e+02,
       4.0500e+02, 2.8000e+02, 2.0600e+02, 1.8000e+02, 1.1500e+02,
       1.0500e+02, 8.3000e+01, 6.1000e+01, 3.9000e+01, 5.3000e+01])
Coordinates:
  * bin      (bin) float64 0.0 6.667e-08 1.333e-07 ... 8e-07 8.667e-07 9.333e-07
Attributes: (12/28)
    paramId:                    172228
    dataType:                   fc
    numberOfPoints:             64144000
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    units_fixed:                1
    regridded:                  1
    edition:                    1
    centre:                     ecmf
    centreDescription:          European Centre for Medium-Range Weather Fore...
    subCentre:                  0

In [30]:
hist_IFS_trop  = diag.hist1d_fast(IFS_chunk_regrided_trop,  preprocess = False)
hist_IFS_trop

<xarray.DataArray (bin: 15)>
array([7.302e+03, 5.970e+02, 2.940e+02, 2.290e+02, 1.450e+02, 1.130e+02,
       7.900e+01, 5.700e+01, 5.500e+01, 3.800e+01, 2.800e+01, 1.400e+01,
       1.200e+01, 7.000e+00, 6.000e+00])
Coordinates:
  * bin      (bin) float64 0.0 6.667e-08 1.333e-07 ... 8e-07 8.667e-07 9.333e-07
Attributes: (12/28)
    paramId:                    172228
    dataType:                   fc
    numberOfPoints:             64144000
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    units_fixed:                1
    regridded:                  1
    edition:                    1
    centre:                     ecmf
    centreDescription:          European Centre for Medium-Range Weather Fore...
    subCentre:                  0

## 3) Diagnostic with automatic data preprocessing

In [ ]:
diag = TR_PR_Diagnostic()

# mandatory attributes
diag.num_of_bins = 15
diag.first_edge = 0
diag.width_of_bin = 1*10**(-4)/diag.num_of_bins

# other attributes of class whoch have a default value
diag.trop_lat = 10
diag.i_timestep = 10
diag.f_timestep = 20

### OR

In [38]:
diag = TR_PR_Diagnostic(num_of_bins = 15, first_edge = 0, width_of_bin = 1*10**(-6)/15, i_timestep = 10, f_timestep = 20)

In [40]:
hist_IFS_prepr  = diag.hist1d_fast(IFS)
hist_IFS_prepr 

KeyError: 'Indexing with a boolean dask array is not allowed. This will result in a dask array of unknown shape. Such arrays are unsupported by Xarray.Please compute the indexer first using .compute()'

## Expected calculation time for entire dataset

In [26]:
import src.time_estimation

In [32]:
importlib.reload(src.time_estimation)
from  src.time_estimation import  estimated_total_calc_time,   optimal_amout_of_timesteps

###  Checking the calculation time of a small chunk of the dataset

In [37]:
result = %timeit -r 2 -n 1 -o  diag.hist1d_fast(IFS_chunk_regrided_trop,  preprocess = False)
time_fast_reg, time_fast_reg_stdev =  result.average, result.stdev 

450 µs ± 72 µs per loop (mean ± std. dev. of 2 runs, 1 loop each)


### Estemated calculation time of entire dataset

In [39]:
estimated_total_calc_time(ds_part = IFS_chunk_regrided_trop, calc_time = time_fast_reg, ds_full = IFS,  units='D')

(86668.67938164828, 1.0031097150653736)

### Optimal amount of timesteps

In [41]:
optimal_amout_of_timesteps(ds_part = IFS_chunk_regrided_trop, calc_time = time_fast_reg,  ds_full = IFS, wanted_time ='3H')

(0.12461249066045943, 673)

## Plots and animations

### Plot of obtained histogram 

In [ ]:
diag.hist_plot(hist_IFS, pdf = False, smooth = False, label='IFS/IFS', plot_title = "ICON, precipitation rate for one timestep")

In [ ]:
diag.hist_plot(hist_IFS_trop, pdf = False, smooth = False, label='IFS/IFS_trop', plot_title = "ICON, trop precipitation rate for one timestep")

### Plot of mean and median value of precipitation rate

In [90]:
diag.mean_and_median_plot(IFS_chunk_regrided, savelabel = 'IFS/IFS')

In [ ]:
diag.mean_and_median_plot(IFS_chunk_regrided_trop, savelabel = 'IFS/IFS_trop')

### Image

In [110]:
image_creator(IFS_chunk_regrided, vmin =0, vmax = 10**(-7),  trop_lat = 90,  label='IFS/IFS')

Done!


In [111]:
image_creator(IFS_chunk_regrided_trop, vmin =0, vmax = 10**(-7),  label='IFS/IFS_trop')

Done!


### Animation

In [34]:
IFS_chunk_regrided = reader.regrid(IFS["tprate"][10:30,:])
IFS_chunk_regrided = IFS_chunk_regrided.compute()

In [52]:
animation_creator(IFS_chunk_regrided, vmin =0, vmax = 10**(-7),   nSeconds = 20,  trop_lat = 90,  label='IFS_tprate')

.....................Done!


True

In [35]:
IFS_chunk_regrided_trop = diag.ds_per_lat_range(IFS_chunk_regrided)
IFS_chunk_regrided_trop = IFS_chunk_regrided_trop.compute()

In [36]:
animation_creator(IFS_chunk_regrided_trop, vmin =0, vmax = 10**(-7), nSeconds = 20,  trop_lat = 10,  label='IFS_tprate_trop')

.....................Done!


True